# Geometrical relaxations from pretrained m3gnet model

In [1]:
import ML_library as MLL
import matgl
import warnings

from pymatgen.io.vasp.inputs import Poscar
from __future__              import annotations
from pymatgen.core           import Structure
from matgl.ext.ase           import M3GNetCalculator, Relaxer
from pymatgen.io.ase         import AseAtomsAdaptor

# To suppress warnings for clearer output
warnings.simplefilter('ignore')

In [2]:
# Define paths to pretrained model and structure to be relaxed

# Materials Project pretrained model as default
model_load_path = 'finetuned_model'
model_load_path = 'M3GNet-MP-2021.2.8-PES' if model_load_path is None else model_load_path

path_to_POSCAR  = 'examples'

# Structure Relaxation

To perform structure relaxation, we use the Relaxer class.

In [3]:
# Load the structure to be relaxed
atoms_ini = Structure.from_file(f'{path_to_POSCAR}/POSCAR')

In [6]:
# Clear caché of previous model
matgl.clear_cache()

# Load the default pre-trained model
#pot = matgl.load_model(path='M3GNet-MP-2021.2.8-PES')
#pot.model.load(model_load_path)
#relaxer = Relaxer(potential=pot)
pot = matgl.load_model(path=model_load_path)
#pot.model.load(model_load_path)
relaxer = Relaxer(potential=pot)

In [7]:
# Relax the structure
relax_atoms_ini = relaxer.relax(atoms_ini, verbose=True)
atoms = relax_atoms_ini['final_structure']

      Step     Time          Energy         fmax
*Force-consistent energies used in optimization.
FIRE:    0 19:35:34      -60.460709*      26.5589
FIRE:    1 19:35:34      -55.440250*      67.7939
FIRE:    2 19:35:34      -60.466656*      26.5406
FIRE:    3 19:35:34      -60.967625*       9.2131
FIRE:    4 19:35:34      -60.914856*      10.6447
FIRE:    5 19:35:34      -60.943428*       9.4187
FIRE:    6 19:35:34      -60.987301*       7.0617
FIRE:    7 19:35:35      -61.025494*       3.7696
FIRE:    8 19:35:35      -61.038826*       1.5587
FIRE:    9 19:35:35      -61.038937*       1.5364
FIRE:   10 19:35:35      -61.039150*       1.4925
FIRE:   11 19:35:35      -61.039459*       1.4284
FIRE:   12 19:35:35      -61.039860*       1.3466
FIRE:   13 19:35:35      -61.040340*       1.2502
FIRE:   14 19:35:36      -61.040890*       1.1431
FIRE:   15 19:35:36      -61.041504*       1.0301
FIRE:   16 19:35:36      -61.042255*       0.9039
FIRE:   17 19:35:36      -61.043167*       0.7701
FI

In [8]:
# Save the relaxed structure as a POSCAR file
poscar_relaxed = Poscar(atoms)
poscar_relaxed.write_file(f'{path_to_POSCAR}/CONTCAR')

In [ ]:
# Or as implemented
poscar_relaxed = MLL.structural_relaxation(path_to_POSCAR, model_load_path)

# Single-shot energy calculation

Perform a single-shot calculation for final structure using M3GNetCalculator.

In [ ]:
# Define the M3GNet calculator
calc = M3GNetCalculator(pot)

# Load atoms adapter and adapt structure
ase_adaptor = AseAtomsAdaptor()
adapted_atoms = ase_adaptor.get_atoms(atoms)

# Calculate potential energy
adapted_atoms.set_calculator(calc)
print(f'The calculated potential energy is {float(adapted_atoms.get_potential_energy()):.3f} eV/supercell')

In [ ]:
# Or as implemented
ssc_energy = MLL.single_shot_energy_calculations(f'{path_to_POSCAR}/CONTCAR', model_load_path)
print(f'The calculated potential energy is {ssc_energy:.3f} eV/supercell')

In [ ]:
# Try on 3x3x3 supercell
ssc_energy = MLL.single_shot_energy_calculations(f'{path_to_POSCAR}/333_supercell/POSCAR', model_load_path)
print(f'The calculated potential energy is {ssc_energy:.3f} eV/supercell')